In [1]:
# Discriminator Model

import torch
import torch.nn as nn

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=2):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride,
                      bias=False, padding_mode="reflect", padding=1),
            nn.InstanceNorm2d(out_channels, affine=True), # Changed from BatchNorm2d - use in both clocks to get better results, as the authors switched out in the CycleGAN paper
            nn.LeakyReLU(0.2),
        )
        
    def forward(self, x):
      return self.conv(x)
    

# Send in sat image, real image => x , y <== conact these along channels
class Discriminator(nn.Module):
	def __init__(self, in_channels=3, features=[64, 128, 256, 512], out_channels=1):
		super().__init__()
		self.initial = nn.Sequential(
			nn.Conv2d(in_channels*2, features[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
			nn.LeakyReLU(0.2),
		)
  
		layers = []
		in_channels = features[0]
		for feature in features[1:]:
			layers.append(
				CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
			)
			in_channels = feature
   
		layers.append(
			nn.Conv2d(
				in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect",
			)
		)
		
		self.model = nn.Sequential(*layers)
  
	def forward(self, x, y):
		x = torch.cat([x,y], dim=1)
		x = self.initial(x)
		return self.model(x)

def test():
  x = torch.rand((1, 3, 256, 256))
  y = torch.rand((1, 3, 256, 256))
  model = Discriminator()
  preds = model(x,y)
  print(preds.shape)
  
test()

torch.Size([1, 1, 30, 30])


In [2]:
# Generator

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1,
                      bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            # Also changed from BatchNorm2d
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2)
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        self.down1 = Block(features, features*2, down=True,
                           act="Leaky", use_dropout=False)  # 64
        self.down2 = Block(features*2, features*4, down=True,
                           act="Leaky", use_dropout=False)  # 32
        self.down3 = Block(features*4, features*8, down=True,
                           act="Leaky", use_dropout=False)  # 16
        self.down4 = Block(features*8, features*8, down=True,
                           act="Leaky", use_dropout=False)  # 8
        self.down5 = Block(features*8, features*8, down=True,
                           act="Leaky", use_dropout=False)  # 4
        self.down6 = Block(features*8, features*8, down=True,
                           act="Leaky", use_dropout=False)  # 2
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1,
                      padding_mode="reflect"), nn.ReLU(),  # 1x1
        )
        self.up1 = Block(features*8, features*8, down=False,
                         act="relu", use_dropout=True)
        self.up2 = Block(features*8*2, features*8, down=False,
                         act="relu", use_dropout=True)
        self.up3 = Block(features*8*2, features*8, down=False,
                         act="relu", use_dropout=True)
        self.up4 = Block(features*8*2, features*8, down=False,
                         act="relu", use_dropout=False)
        self.up5 = Block(features*8*2, features*4, down=False,
                         act="relu", use_dropout=False)
        self.up6 = Block(features*4*2, features*2, down=False,
                         act="relu", use_dropout=False)
        self.up7 = Block(features*2*2, features, down=False,
                         act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, 4, 2, 1),
            nn.Tanh(),
        )

    def forward(self, x):
        # This shape mirrors u-net architecture
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], dim=1))
        up3 = self.up3(torch.cat([up2, d6], dim=1))
        up4 = self.up4(torch.cat([up3, d5], dim=1))
        up5 = self.up5(torch.cat([up4, d4], dim=1))
        up6 = self.up6(torch.cat([up5, d3], dim=1))
        up7 = self.up7(torch.cat([up6, d2], dim=1))
        return self.final_up(torch.cat([up7, d1], dim=1))


def test():
    x = torch.randn((1, 3, 256, 256))
    model = Generator(in_channels=3, features=64)
    gen = model(x)
    print(gen.shape)


test()


torch.Size([1, 3, 256, 256])


In [3]:
# Utils
from torchvision.utils import save_image


def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)
    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
    gen.train()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


In [4]:
# Dataset Loading

from PIL import Image
import numpy as np
import os
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import random

both_transform = A.Compose(
    [A.Resize(width=256, height=256), ], additional_targets={"image0": "image"},
)

transform_only_input = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(p=0.2),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[
                    0.5, 0.5, 0.5], max_pixel_value=255.0),
        ToTensorV2()
    ],
)

transform_only_target = A.Compose(
	[
		A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0),
		ToTensorV2()
	]
)

class MapLoader(Dataset):
    def __init__(self, root_dir, img_size):
        self.img_size = img_size
        self.root_dir = root_dir
        self.list_files = os.listdir(self.root_dir)
        print(self.list_files[:8])

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_file)
        image = np.array(Image.open(img_path))
        input_image = image[:, :self.img_size, :]
        target_image = image[:, self.img_size:, :]
        augmentations = both_transform(
            image=input_image, image0=target_image)
        input_image, target_image = augmentations["image"], augmentations["image0"]
        input_image = transform_only_input(image=input_image)["image"]
        target_image = transform_only_target(image=target_image)["image"]
        
        return input_image, target_image
    
    
class TLLLoader(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.left = os.listdir(self.root_dir + "/left")
        self.right = os.listdir(self.root_dir + "/right")
        print(self.left[:8])

    def __len__(self):
        return len(self.left)

    def __getitem__(self, index):
        img_left = self.left[index]
        img_right = self.right[index]
        img_path_left = os.path.join(self.root_dir + "/left", img_left)
        img_path_right = os.path.join(self.root_dir + "/right", img_right)
        image_l = np.array(Image.open(img_path_left))
        image_r = np.array(Image.open(img_path_right))
        input_image = image_l
        target_image = image_r
        augmentations = both_transform(
            image=input_image, image0=target_image)
        input_image, target_image = augmentations["image"], augmentations["image0"]
        input_image = transform_only_input(image=input_image)["image"]
        target_image = transform_only_target(image=target_image)["image"]
        
        return input_image, target_image




In [5]:
# Train

import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm, trange

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
D_LR = 4e-4
LR = 2e-4
BATCH_SIZE = 64
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
NUM_EPOCHS = 100
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"


def train_fn(disc, gen, loader, opt_disc, opt_gen, l1, bce, g_scaler, d_scaler, epoch):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_fake = disc(x, y_fake.detach())
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1(y_fake, y) * L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        loop.set_postfix(e=str(epoch)+"/"+str(NUM_EPOCHS),
                         D_loss=D_loss.item(), G_loss=G_loss.item())


def main():
    disc = Discriminator(in_channels=CHANNELS_IMG).to(DEVICE)
    gen = Generator(in_channels=CHANNELS_IMG).to(DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=D_LR, betas=(0.5, 0.999))
    opt_gen = optim.Adam(gen.parameters(), lr=LR, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if LOAD_MODEL:
        load_checkpoint(CHECKPOINT_GEN, gen, opt_gen, LR)
        load_checkpoint(CHECKPOINT_DISC, disc, opt_disc, LR)

    train_dataset = MapLoader(root_dir="./../_datasets/cityscapes_data/train", img_size=IMAGE_SIZE)
    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()
    val_dataset = MapLoader(root_dir="./../_datasets/cityscapes_data/val", img_size=IMAGE_SIZE)
    val_loader = DataLoader(val_dataset, batch_size=32,
                            shuffle=True)

    for epoch in range(NUM_EPOCHS):
        train_fn(disc, gen, train_loader, opt_disc,
                 opt_gen, L1_LOSS, BCE, g_scaler, d_scaler, epoch)

        if SAVE_MODEL and epoch % 5 == 0:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
            save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)

        save_some_examples(gen, val_loader, epoch, folder="image")


main()


['1.jpg', '10.jpg', '100.jpg', '1000.jpg', '1001.jpg', '1002.jpg', '1003.jpg', '1004.jpg']
['1.jpg', '10.jpg', '100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg']


100%|██████████| 47/47 [05:57<00:00,  7.60s/it, D_loss=0.232, G_loss=26.9, e=0/100] 


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 47/47 [03:02<00:00,  3.88s/it, D_loss=0.443, G_loss=20.3, e=5/100] 


=> Saving checkpoint
=> Saving checkpoint


...

100%|██████████| 47/47 [02:55<00:00,  3.74s/it, D_loss=0.0294, G_loss=15.3, e=90/100] 


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 47/47 [02:59<00:00,  3.81s/it, D_loss=0.658, G_loss=11.6, e=95/100]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 47/47 [03:02<00:00,  3.88s/it, D_loss=0.481, G_loss=14.6, e=99/100] 
